In [1]:
!pip install beautifulsoup4 requests
!pip install schedule
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [14]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep
import schedule
import pandas as pd
import re
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import spacy
import re

import numpy as np
import pandas as pd
from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sb

from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import scipy.stats as stats

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
output_notebook()

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sb

from textblob import TextBlob
import scipy.stats as stats
from datetime import datetime

In [15]:
pagesToGet= 1
upperframe=[]  
submeu_li = []
n_topics = 50

nlp=spacy.load('en_core_web_sm')
all_stopwords = nlp.Defaults.stop_words

In [16]:
#Get Soup
def get_soup(url1):

  page=""
  #an exception might be thrown, so the code should be in a try-except block
  try:
      #use the browser to get the url. This is suspicious command that might blow up.
      page=requests.get(url1)                             # this might throw an exception if something goes wrong.

  except Exception as e:                                   # this describes what to do if an exception is thrown
      error_type, error_obj, error_info = sys.exc_info()      # get the exception information
      print ('ERROR FOR LINK:',url1)                          #print the link that cause the problem
      print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
  
  soup=BeautifulSoup(page.text,'html.parser')
  return soup

In [17]:
#Fetch news
def get_news(): 
  url = "https://www.npr.org/sections/news/" 
  frame =[]
  soup = get_soup(url)
  links=soup.find_all(class_="item")
  
  for j in links:
    St = j.find("div",attrs={'class':'item-info'}).find("h2").text.strip()
    La = j.find("p").text.strip()
    if(len(La) > 10 and len(St) > 10):
      frame.append((St + " " +La))
    
  submeu=soup.find(class_="animated")#.find_all("li")

  for a in submeu.find_all('a', href=True):
    url = "https://www.npr.org" + a['href']
    soup = get_soup(url)
    links=soup.find_all(class_="item")
    for j in links:
      Statement = j.find("div",attrs={'class':'item-info'}).find("h2").text.strip()
      Label = j.find("p").text.strip()
      if(len(Label) > 5 and len(Statement) > 5):
        frame.append((Statement+ " " +Label))

  frame = [re.sub('\n','', f) for f in frame]
  frame = [re.sub(' +', ' ', f) for f in frame]
  frame = list(set(frame))

  return frame

In [18]:
#remove stopwords
def remove_mystopwords(sentence):
    tokens = sentence.split(" ")
    tokens_filtered= [word for word in tokens if not word in all_stopwords]
    return (" ").join(tokens_filtered)

In [19]:
#Clean data
def data_cleaning(frame):
  en_core = spacy.load('en_core_web_sm')
  #print("++++++++", frame)
  lemma_frame_ls = []
  frame = [f.lower() for f in frame]
  frame = [re.sub('hide caption','', word) for word in frame]
  frame = [re.sub('getty images','', word) for word in frame]
  frame = [re.sub('[^A-Za-z ]+', '', word) for word in frame]
  #print("--------------", frame)
  frame =[remove_mystopwords(text) for text in frame]
  frame = [re.sub(' +', ' ', word) for word in frame]

  df = pd.DataFrame(frame, columns=['data'])
  df["data_cleanned"] = df['data'].apply(lambda x: " ".join([y.lemma_ for y in en_core(x)]))

  return df

In [20]:
# Define helper functions
def get_top_n_words(n_top_words, count_vectorizer, text_data):
    '''
    returns a tuple of the top n words in a sample and their 
    accompanying counts, given a CountVectorizer object and text sample
    '''
    vectorized_headlines = count_vectorizer.fit_transform(text_data.values)
    vectorized_total = np.sum(vectorized_headlines, axis=0)
    word_indices = np.flip(np.argsort(vectorized_total)[0,:], 1)
    word_values = np.flip(np.sort(vectorized_total)[0,:],1)
    
    word_vectors = np.zeros((n_top_words, vectorized_headlines.shape[1]))
    for i in range(n_top_words):
        word_vectors[i,word_indices[0,i]] = 1

    words = [word[0].encode('ascii').decode('utf-8') for 
             word in count_vectorizer.inverse_transform(word_vectors)]

    return (words, word_values[0,:n_top_words].tolist()[0])

In [21]:
# Define helper functions
def get_keys(topic_matrix):
    '''
    returns an integer list of predicted topic 
    categories for a given topic matrix
    '''
    keys = topic_matrix.argmax(axis=1).tolist()
    return keys

def keys_to_counts(keys):
    '''
    returns a tuple of topic categories and their 
    accompanying magnitudes for a given list of keys
    '''
    count_pairs = Counter(keys).items()
    categories = [pair[0] for pair in count_pairs]
    counts = [pair[1] for pair in count_pairs]
    return (categories, counts)

In [22]:
# Define helper functions
def get_top_n_words(n, keys, document_term_matrix, count_vectorizer):
    '''
    returns a list of n_topic strings, where each string contains the n most common 
    words in a predicted category, in order
    '''
    top_word_indices = []
    for topic in range(n_topics):
      try:
        temp_vector_sum = 0
        for i in range(len(keys)):
            if keys[i] == topic:
                temp_vector_sum += document_term_matrix[i]
        if not isinstance(temp_vector_sum, int):
          temp_vector_sum = temp_vector_sum.toarray()
          top_n_word_indices = np.flip(np.argsort(temp_vector_sum)[0][-n:],0)
          top_word_indices.append(top_n_word_indices) 
      except:
        continue  
    top_words = []
    for topic in top_word_indices:
        topic_words = []
        for index in topic:
            temp_word_vector = np.zeros((1,document_term_matrix.shape[1]))
            temp_word_vector[:,index] = 1
            the_word = count_vectorizer.inverse_transform(temp_word_vector)[0][0]
            topic_words.append(the_word.encode('ascii').decode('utf-8'))
        top_words.append(" ".join(topic_words))         
    return top_words

In [23]:
#Implement LDA
def impl_lda():
  reindexed_data = data_cleaning(get_news())
  reindexed_data = reindexed_data['data_cleanned']
  count_vectorizer = CountVectorizer(stop_words='english', max_features=40000)

  print('Headline before vectorization: {}'.format(reindexed_data[123]))

  document_term_matrix = count_vectorizer.fit_transform(reindexed_data)

  lda_model = LatentDirichletAllocation(n_components=n_topics, learning_method='online', 
                                            random_state=0, verbose=0)
  lda_topic_matrix = lda_model.fit_transform(document_term_matrix)

  lda_keys = get_keys(lda_topic_matrix)
  lda_categories, lda_counts = keys_to_counts(lda_keys)
  top_3_words_lda = get_top_n_words(3, lda_keys, document_term_matrix, count_vectorizer)
  #for t in range(n_topics):
      #print(top_3_words_lda[t])
  return top_3_words_lda

In [24]:
#Save Topic in CSV file
def save_tiopic():
  top_3_words_lda = impl_lda()
  now = datetime.now()
  current_time = now.strftime("%H_%M_%S")
  print("Current thread execution time =", current_time)
  
  
  print("Current Topic: ", top_3_words_lda)
  df = pd.DataFrame(top_3_words_lda)
  df.to_csv(f"a_{current_time}.csv", index = False, header=True)

In [28]:
  now = datetime.now()
  current_time = now.strftime("%H_%M_%S")
  print("Current thread execution time =", current_time)

Current thread execution time = 06_23_36


In [25]:
#Scheduler
def schedule_actions():
  schedule.every().day.at('06:11:27').do(save_tiopic)
  #schedule.every(1).minutes.do(save_tiopic)
  while True:
    schedule.run_pending()
    sleep(1)

schedule_actions()

Headline before vectorization: biden ask congress billion aid ukraine war drag president joe biden speak war ukraine roosevelt room white house thursday andrew harnikap
Current thread execution time = 06_05_22
Current Topic:  ['oyster texas johny', 'figure dance black', 'donnella summer demby', 'lgbtq youth care', 'australian porpoise minister', 'astronaut mariupol azovstal', 'sample state clot', 'senate ohio primary', 'kasha fairfax hear', 'code abortion hari', 'quantitative uranus tightening', 'texas solar fossil', 'detect test ba', 'new european eu', 'climate scotland dam', 'climate air carbon', 'president rep biden', 'ohio primary senate', 'mccarthy vaccine house', 'climate food change', 'pron emission perform', 'abortion supreme court', 'arizona fit judge', 'trump republican speak', 'shirt wear hold', 'twin alex spektor', 'gas harris april', 'bassist carter breed', 'internet npr player', 'mayorkas secretary dhs', 'new democratic variant', 'zurich gas chicago', 'new india million',

KeyboardInterrupt: ignored